In [1]:
import numpy as np
import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense, Embedding, Lambda
from keras import utils
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
import gensim

In [2]:
data=open('corona.txt','r')

In [3]:
corona_data = [text for text in data if text.count(' ') >= 2]

In [4]:
vectorize = Tokenizer()
vectorize.fit_on_texts(corona_data)

In [5]:
corona_data = vectorize.texts_to_sequences(corona_data)

In [6]:
total_vocab = sum(len(s) for s in corona_data)
word_count = len(vectorize.word_index) + 1
window_size = 2

In [7]:
def cbow_model(data, window_size, total_vocab):
    total_length = window_size*2
    for text in data:
        text_len = len(text)
        for idx, word in enumerate(text):
            context_word = []
            target   = []
            begin = idx - window_size
            end = idx + window_size + 1
            context_word.append([text[i] for i in range(begin, end) if 0 <= i < text_len and i != idx])
            target.append(word)
            contextual = sequence.pad_sequences(context_word, total_length=total_length)
            final_target = utils.to_categorical(target, total_vocab)
            yield(contextual, final_target)

In [8]:
model = Sequential()
model.add(Embedding(input_dim=total_vocab, output_dim=100, input_length=window_size*2))
model.add(Lambda(lambda x: K.mean(x, axis=1), output_shape=(100,)))
model.add(Dense(total_vocab, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
for i in range(10):
    cost = 0
    for x, y in cbow_model(data, window_size, total_vocab):
        cost += model.train_on_batch(contextual, final_target)
    print(i, cost)


0 0
1 0
2 0
3 0
4 0
5 0
6 0
7 0
8 0
9 0


In [9]:
dimensions=100
vect_file = open('vectors.txt','w')
vect_file.write('{} {}\n'.format(101,dimensions))


8

In [10]:
weights = model.get_weights()[0]
for text, i in vectorize.word_index.items():
    final_vec = ' '.join(map(str, list(weights[i, :])))
    vect_file.write('{} {}\n'.format(text, final_vec))
vect_file.close()

In [11]:
cbow_output = gensim.models.KeyedVectors.load_word2vec_format('vectors.txt', binary=False)
cbow_output.most_similar(positive=['virus'])

[('infection', 0.283257395029068),
 ('higher', 0.27101564407348633),
 ('before', 0.24137583374977112),
 ('major', 0.23302049934864044),
 ('does', 0.2174900472164154),
 ('time', 0.15912482142448425),
 ('important', 0.15492355823516846),
 ('shorter', 0.14368218183517456),
 ('cases', 0.13826143741607666),
 ('interval', 0.11918780207633972)]